In [1]:
import pandas as pd
from sqlalchemy import create_engine
from time import time

In [2]:
pd.__version__

'2.2.2'

# Parameters

In [3]:
filename = 'yellow_tripdata_2021-01.csv'
database_user = 'root'
database_pwd = 'root'
database_host = 'localhost'
database_db = 'ny_taxi'
database_port = 5432

In [4]:
filename = 'yellow_tripdata_2021-01.csv'
df = pd.read_csv(filename, nrows=1000, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
df.name = "_".join(filename.split("_")[:2])

In [8]:
engine = create_engine(f'postgresql://{database_user}:{database_pwd}@{database_host}:{database_port}/{database_db}')

In [9]:
engine.connect()

In [10]:
print(pd.io.sql.get_schema(df, name=df.name))

CREATE TABLE "yellow_tripdata" (
"VendorID" REAL,
  "tpep_pickup_datetime" TIMESTAMP,
  "tpep_dropoff_datetime" TIMESTAMP,
  "passenger_count" REAL,
  "trip_distance" REAL,
  "RatecodeID" REAL,
  "store_and_fwd_flag" REAL,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" REAL,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL
)


In [11]:
df_iter = pd.read_csv(filename, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], iterator=True, chunksize=100000)

In [12]:
df = next(df_iter)
df.name = "_".join(filename.split("_")[:2])
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,0.20,1,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2021-01-04 14:04:31,2021-01-04 14:08:52,3,0.70,1,N,234,224,2,5.0,2.5,0.5,0.00,0.0,0.3,8.30,2.5
99996,1,2021-01-04 14:18:46,2021-01-04 14:35:45,2,3.30,1,N,234,236,1,14.5,2.5,0.5,3.55,0.0,0.3,21.35,2.5
99997,1,2021-01-04 14:42:41,2021-01-04 14:59:22,2,4.70,1,N,236,79,1,17.0,2.5,0.5,4.05,0.0,0.3,24.35,2.5
99998,2,2021-01-04 14:39:02,2021-01-04 15:09:37,2,17.95,2,N,132,148,1,52.0,0.0,0.5,5.00,0.0,0.3,60.30,2.5


In [13]:
df.head(n=0).to_sql(name=df.name, con=engine, if_exists='replace')

0

In [14]:
%time df.to_sql(name=df.name, con=engine, if_exists='append')

CPU times: user 7.47 s, sys: 137 ms, total: 7.61 s
Wall time: 12.1 s


1000

In [15]:
while True:
    t_start = time()
    df = next(df_iter)
    df.name = "_".join(filename.split("_")[:2])

    %time df.to_sql(name=df.name, con=engine, if_exists='append')

    t_end = time()

    print('inserted another chunk..., took %.3f seconds' % (t_end - t_start))

CPU times: user 7.48 s, sys: 64 ms, total: 7.55 s
Wall time: 12 s
inserted another chunk..., took 12.330 seconds
CPU times: user 7.51 s, sys: 44.1 ms, total: 7.55 s
Wall time: 12.1 s
inserted another chunk..., took 12.371 seconds
CPU times: user 7.49 s, sys: 67.7 ms, total: 7.56 s
Wall time: 12.1 s
inserted another chunk..., took 12.365 seconds
CPU times: user 7.48 s, sys: 72.4 ms, total: 7.55 s
Wall time: 12.1 s
inserted another chunk..., took 12.344 seconds
CPU times: user 7.48 s, sys: 51.7 ms, total: 7.53 s
Wall time: 12.1 s
inserted another chunk..., took 12.361 seconds
CPU times: user 7.53 s, sys: 15.9 ms, total: 7.55 s
Wall time: 12 s
inserted another chunk..., took 12.303 seconds
CPU times: user 7.46 s, sys: 36.1 ms, total: 7.5 s
Wall time: 12 s
inserted another chunk..., took 12.335 seconds
CPU times: user 7.56 s, sys: 24.1 ms, total: 7.59 s
Wall time: 12.1 s
inserted another chunk..., took 12.355 seconds
CPU times: user 7.7 s, sys: 32.4 ms, total: 7.74 s
Wall time: 12.3 s
inse

/tmp/ipykernel_13882/430744713.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = next(df_iter)


CPU times: user 7.46 s, sys: 28 ms, total: 7.49 s
Wall time: 11.9 s
inserted another chunk..., took 12.196 seconds
CPU times: user 4.6 s, sys: 20 ms, total: 4.62 s
Wall time: 7.4 s
inserted another chunk..., took 7.592 seconds


StopIteration: 